In [10]:
import ast
import json
import ijson
from tqdm import tqdm 

In [11]:
with open("/auto/home/knarik/Molecular_Generation_with_GDB13/src/ablations/generations/generations/code/clusters.json") as f:
    clusters = json.load(f)  

In [12]:
len(clusters)

656656

In [13]:
def get_bytecode(code_str):
    """Compile code string to bytecode."""
    code = ast.parse(code_str)
    
    # Bytecode Object
    bytecode = compile(code, '<string>', 'exec')

    # Bytecode bytes
    bytecode = bytecode.co_code  

    return bytecode

In [14]:
class CanonicalizeCommutativeOps(ast.NodeTransformer):
    def visit_BinOp(self, node):
        self.generic_visit(node)

        if isinstance(node.op, (ast.Add, ast.Mult)):
            operands = self.flatten_commutative_chain(node, type(node.op))
            sorted_operands = sorted(operands, key=self.sort_key)

            return self.build_commutative_chain(sorted_operands, node.op)

        return node

    def flatten_commutative_chain(self, node, op_type):
        if isinstance(node, ast.BinOp) and isinstance(node.op, op_type):

            return self.flatten_commutative_chain(node.left, op_type) + \
                   self.flatten_commutative_chain(node.right, op_type)
        else:
            return [node]

    def build_commutative_chain(self, operands, op):
        if not operands:
            return ast.Constant(value=0 if isinstance(op, ast.Add) else 1)

        expr = operands[0]

        for operand in operands[1:]:
            expr = ast.BinOp(left=expr, op=op, right=operand)

        return expr

    def sort_key(self, node):
        if isinstance(node, ast.Constant):
            return (0, repr(node.value))
        elif isinstance(node, ast.Name):
            return (1, node.id)
        else:
            return (2, ast.dump(node))


def normalize_code(code):
    tree = ast.parse(code)
    normalized = CanonicalizeCommutativeOps().visit(tree)
    ast.fix_missing_locations(normalized)

    return ast.unparse(normalized)

In [15]:

with open("/auto/home/knarik/Molecular_Generation_with_GDB13/src/data/data/data_bin_python/valid/val.jsonl", "r") as f_val:
    with open("/auto/home/knarik/Molecular_Generation_with_GDB13/src/ablations/generations/generations/code/random_output.jsonl", "a") as f_out:
        for line in tqdm(f_val):
            data = json.loads(line)
            code_line = data["text"] 
            
            normalized_line = normalize_code(code_line)
            bytecode = get_bytecode(str(normalized_line))

            for item in clusters[str(bytecode)]:
                json_obj = {"text": item}
                f_out.write(json.dumps(json_obj) + "\n")

0it [00:00, ?it/s]

10000it [00:02, 3461.50it/s]
